## SageMaker built-in algorithm (Image classification)

SageMaker에서는 여러가지 built-in 알고리즘을 제공하고 있습니다.
- built-in 종류 : https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html

그 중에서 이번에는 MobileNet 기반의 알고리즘을 활용해서 image classification 해 보도록 하겠습니다.
- image classification w/ TF
  - 문서 : https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification-tensorflow.html 
  - 예시 노트북 : https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/image_classification_tensorflow/Amazon_TensorFlow_Image_Classification.ipynb


In [ ]:
# !pip install -q sagemaker ipywidgets --upgrade

In [ ]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker.get_execution_role()
print(aws_role)

sagemaker_session = Session()
bucket = sagemaker_session.default_bucket()
print(bucket)

In [ ]:
model_id, model_version = "tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4", "*"

In [ ]:
import IPython
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models
from sagemaker.jumpstart.filters import And

# Retrieves all TensorFlow Image Classification models made available by SageMaker Built-In Algorithms.
filter_value = And("task == ic", "framework == tensorflow")
ic_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown, for user to select a model.
dropdown = Dropdown(
    options=ic_models,
    value=model_id,
    description="SageMaker Built-In TensorFlow Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(IPython.display.Markdown("### 아래에서 사용할 pre-trained 모델을 선택해 주세요"))
display(dropdown)

In [ ]:
print(dropdown.value)

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = dropdown.value, "*"
training_instance_type = "ml.g4dn.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

In [ ]:
training_dataset_s3_path = f"s3://{bucket}/lowcode-sm/wwi-dataset/train/"
s3_output_location = f"s3://{bucket}/lowcode-sm/logs/image-classification-output/"

In [ ]:
print(training_dataset_s3_path)
print(s3_output_location)

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epochs"] = "8"
print(hyperparameters)

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"wwi-image-classification-{model_id}-transfer-learning")

training_metric_definitions = [
    {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
    {"Name": "train_accuracy", "Regex": "- accuracy: ([0-9\\.]+)"},
    {"Name": "train_loss", "Regex": "- loss: ([0-9\\.]+)"},
]

# Create SageMaker Estimator instance
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    metric_definitions=training_metric_definitions,
)

In [ ]:
ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

### 학습에 걸리는 시간 예시

- MobileNet (tensorflow-ic-imagenet-mobilenet-v2-100-224-classification-4) 5 epoch 학습하는 데 g4dn.2xlarge 에서 5분 이하로 걸립니다.
- 위의 모델 10epoch의 경우 330 sec (5.5분) 정도 걸리게 됩니다.
- 만일 더 좋은 성능을 내는 모델, 예를들어 EfficientNet (tensorflow-ic-efficientnet-v2-imagenet1k-m) 10 epoch을 학습하는 경우 30분 가까이 걸리게 됩니다.

In [ ]:
inference_instance_type = "ml.m5.large"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

In [ ]:
endpoint_name = name_from_base(f"wwi-classifier-{model_id}-")

In [ ]:
predictor = ic_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

In [ ]:
from IPython.core.display import HTML

In [ ]:

def get_predict_output(image_path):

    with open(image_path, "rb") as file:
        img = file.read()

    query_response = predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]

    display(
        HTML(
            f'<img src={image_path} alt={image_path} align="left" style="width: 250px;"/>'
            f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
        )
    )


In [ ]:
import os
import random
test_dir = "data/wwi-dataset/test"
label_list = os.listdir(test_dir)

In [ ]:
%%time
for label in label_list:
    test_images = os.listdir(os.path.join(test_dir, label))
    random.shuffle(test_images)
    img_path = os.path.join(test_dir, label, test_images[0])
    get_predict_output(img_path)